In [0]:
%run ./project_config

In [0]:
%run ./parameters

In [0]:
from pyspark.sql import functions as f, DataFrame
from pyspark.sql.window import Window
from functions import load_table, save_table, read_csv_file, map_column_values
from functions.functions import union_dataframe_list

# Load table

In [0]:
cohort_filtered = load_table('cohort_filtered')
display(cohort_filtered.limit(100))

primary_care_meds = load_table('primary_care_meds', method='primary_care_meds')
display(primary_care_meds.limit(100))


# 2 Codelists

In [0]:
dict_codelists_bnf = {
    "antihypertensives": "./codelists/antihypertensives_bnf.csv",
    "insulin": "./codelists/insulin_bnf.csv",
}

list_codelists_bnf = [
    read_csv_file(codelist_path)
    .withColumn('phenotype', f.lit(phenotype))
    for phenotype, codelist_path in dict_codelists_bnf.items()
]

codelist_bnf = spark.createDataFrame(union_dataframe_list(list_codelists_bnf))

display(codelist_bnf)


# 3 Prepare datasets

In [0]:
primary_care_meds = (
    primary_care_meds
    .select(
        'person_id',
        f.col('prescribedbnfcode').alias('code'),
        f.col('processingperioddate').alias('date'),
        f.lit('primary_care_meds').alias('data_source'),
        f.lit(1).alias('source_priority')
    )
)

# 4 Cohort dates

In [0]:
cohort_prepared_meds = (
    cohort_filtered
    .select(
        'person_id',
        f.date_sub('date_of_birth', 2*365).alias('min_date'),
        f.date_sub('index_mi_date', 0).alias('max_date')
    )
)

# 5 Perform match

In [0]:
primary_care_meds_matched = (
    primary_care_meds
    .join(
        f.broadcast(codelist_bnf),
        on='code', how='inner'
    )
    .join(
        cohort_prepared_meds,
        on='person_id', how='inner'
    )
    .filter("(date >= min_date) AND (date <= max_date)")
)

In [0]:
medications_all_events = (
    primary_care_meds_matched
)

save_table(medications_all_events, 'medications_all_events')

# 6 Aggregate and pivot wider

In [0]:
medications_all_events = load_table('medications_all_events')

_win = Window.partitionBy('person_id', 'phenotype').orderBy(f.col('date').desc(), 'source_priority')

medications_last_event = (
    medications_all_events
    .withColumn('rank', f.row_number().over(_win))
    .filter('rank = 1')
    .withColumn('flag', f.lit(1))
)

medications_last_event = (
    medications_last_event
    .groupBy('person_id')
    .pivot('phenotype')
    .agg(
        f.first('flag').alias('flag'),
        f.first('date').alias('date'),
        f.first('code').alias('code'),
        f.first('data_source').alias('source')
    )
)

save_table(medications_last_event, 'medications_last_event')

# 7 Save table

In [0]:
cohort_filtered = load_table('cohort_filtered')
medications_last_event = load_table('medications_last_event')

cohort_medications = (
    cohort_filtered
    .select('person_id')
    .join(
        medications_last_event,
        on='person_id', how='left'
    )
)
    
save_table(cohort_medications, 'cohort_medications')

# 8 Display

In [0]:
cohort_medications = load_table('cohort_medications')
display(cohort_medications.limit(100))